# Pré-processamento

Importações

In [1]:
import numpy as np
import pandas as pd
import ast

Carregando dados

In [2]:
credits_path, movies_path = 'files/credits.csv', 'files/movies.csv'
df_credits = pd.read_csv(credits_path)
df_movies = pd.read_csv(movies_path)

Configurando o pandas para mostrar todas as linhas e colunas do DataFrame (útil para DF grandes).

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Colunas antes do merge

In [4]:
print("collumns num: ", len(df_movies.columns), "\nCollumns: ", str(df_movies.columns)[6:-20])

collumns num:  20 
Collumns:  ['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
 


Junção dos dois DataFrames utilizando o 'title' como chave

In [5]:
df_movies = df_movies.merge(df_credits, on='title')

Colunas após o merge

In [6]:
print("collumns num: ", len(df_movies.columns), "\nCollumns: ", str(df_movies.columns)[6:-20])

collumns num:  23 
Collumns:  ['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
 


Selecionando somente as colunas necessárioas para o processamento

In [7]:
columns = ['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']
df_movies = df_movies[columns]

In [8]:
print("collumns num: ", len(df_movies.columns), "\nCollumns: ", str(df_movies.columns)[6:-20])

collumns num:  7 
Collumns:  ['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'cre


Remoção linhas caso tenha valores nulos

In [9]:
if any (df_movies.isnull().sum()):
    df_movies.dropna(axis=0, how='any', inplace=True)
    print('All null values dropped')

else:
    print('Have no Null Values')

All null values dropped


Remoção linhas caso tenha linhas repetidas

In [10]:
if df_movies.duplicated().sum():
    df_movies.dropduplicates(inplace=True)
    print('All null duplicated values dropped')
else:
    print('Have no duplicated values.')

Have no duplicated values.


Funções para formatação dos dados

In [11]:
def convert(obj):
    """
    Retorna lista contendo os nomes em um dicionário
    """
    return [i['name'] for i in ast.literal_eval(obj)]
def convert3(obj):
    """
    Retorna lista contendo os 3 primeiros nomes em um dicionário
    """
    return [j['name'] for i, j in enumerate(ast.literal_eval(obj)[:3])]
def fetch_director(obj):
    """
    Retorna lista contendo somente o nome caso o cargo seja diretor
    """
    return [i['name'] for i in ast.literal_eval(obj) if i['job'] == 'Director']

Aplicação das funções para formatação dos dados

In [12]:
# coluna recebe apenas o nome do gênero
df_movies['genres'] = df_movies['genres'].apply(convert)
# coluna recebe apenas a(s) palavras-chave
df_movies['keywords'] = df_movies['keywords'].apply(convert)
# coluna recebe apenas os 3 primeiros nomes do elenco
df_movies['cast'] = df_movies['cast'].apply(convert3)
# coluna recebe apenas os nomes dos diretores
df_movies['crew'] = df_movies['crew'].apply(fetch_director)

In [13]:
# separa as palavras da descrição do filme (facilita tarefas de NLP)
df_movies['overview'] = df_movies['overview'].apply(lambda x: x.split())

Retira espaços de nomes com mais de uma palavra em colunas específicas

In [14]:
df_movies['genres'] = df_movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
df_movies['crew'] = df_movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])
df_movies['cast'] = df_movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
df_movies['keywords'] = df_movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])

Cria uma coluna contendo as informações das outras colunas

In [15]:
df_movies['tags'] = df_movies['overview'] + df_movies['genres'] + df_movies['keywords'] + df_movies['cast'] + df_movies['crew']

Criação do DataFrame final utilizando somente as colunas necessárias para o sistema

In [16]:
cols = ['movie_id', 'title', 'tags']
new_df = df_movies[cols]

Formatação do conteúdo da coluna tags

In [17]:
# retira os colchetes
new_df['tags'] = new_df['tags'].apply(lambda x : ' '.join(x))
# converte todas as letras para minúsculas
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

/tmp/ipykernel_213763/2503150204.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : ' '.join(x))
/tmp/ipykernel_213763/2503150204.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())


# NLP

Importações

In [18]:
# tokenização e construção de dicionário
from sklearn.feature_extraction.text import CountVectorizer
import nltk
# Stemmin (basicamente remove os sufixos das palavras)
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

Instância da classe de vetorização de texto CountVectorizer

In [19]:
cv = CountVectorizer(max_features=5000, stop_words='english')

Cria o vocabulário e a matriz de contagem de palavras a partir da coluna 'tags'

In [20]:
vectors = cv.fit_transform(new_df['tags']).toarray()

Função para aplicar o stemmin em um texto

In [21]:
ps = PorterStemmer()

def stem(text):
    return " ".join([ps.stem(i) for i in text.split()])

In [22]:
# stemmin em todas as palavras existentes na coluna 'tags'
new_df['tags'] = new_df['tags'].apply(stem)

/tmp/ipykernel_213763/1150880088.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [23]:
# Cria uma matriz contendo a similaridade para cada par de vetor existente
similarity = cosine_similarity(vectors)

# Recommender

Função para encontrar, ordenar e apresentar os filmes recomendados com base na similaridade entre as tags

In [24]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    temp = f"{'='*3} Top 5 recommendations for '{movie}' {'='*3}"
    print(temp)
    for i, j in enumerate(movies_list):
        print(f"{i+1}. ", new_df.iloc[j[0]].title)
    print("=" * len(temp), "\n")

## Test

testa o sistema com os 5 primeiros filmes do DataFrame

In [25]:
movies = new_df['title'][:5]

In [26]:
[recommend(movie) for movie in movies]

=== Top 5 recommendations for 'Avatar' ===
1.  Titan A.E.
2.  Independence Day
3.  Small Soldiers
4.  Aliens vs Predator: Requiem
5.  Krull

=== Top 5 recommendations for 'Pirates of the Caribbean: At World's End' ===
1.  Pirates of the Caribbean: Dead Man's Chest
2.  Pirates of the Caribbean: The Curse of the Black Pearl
3.  Pirates of the Caribbean: On Stranger Tides
4.  20,000 Leagues Under the Sea
5.  Puss in Boots

=== Top 5 recommendations for 'Spectre' ===
1.  Quantum of Solace
2.  Never Say Never Again
3.  Skyfall
4.  From Russia with Love
5.  Thunderball

=== Top 5 recommendations for 'The Dark Knight Rises' ===
1.  The Dark Knight
2.  Batman Begins
3.  Batman
4.  Batman Returns
5.  Batman

=== Top 5 recommendations for 'John Carter' ===
1.  Star Trek: Insurrection
2.  Mission to Mars
3.  Captain America: The First Avenger
4.  Escape from Planet Earth
5.  Ghosts of Mars



[None, None, None, None, None]